# IT44 TP3: Courbes de Bezier et B-splines
---

<table style="background: white;">
    <tr style="background: white;">
        <td> <img src='bezier.png'  width="500px">       
    </tr>
</table>
Dans ce TP vous allez travailler sur l'algorithme de de Casteljau pour tracer des courbes de Bézier et tracer également des fonctions B-splines.

## Algorithme de de Casteljau

Ecrire un fonction ```Casteljau(P_0,..,P_n,t)``` s'appuyant sur l'algorithme de de Castejau qui étant donnée un polygone de contrôle ${\bf P}=(P_0,\dots, P_n)$ et étant donné un temps $t\in [0,1]$, calcule les coordonnées de $P_n ^n(t)$.

## Tracé d'une courbe de Bézier

Utiliser la fonction précédente pour tracer des exemples de courbes de Bézier pour des polygones de contrôle de votre choix.



## Tracé d'une courbe de Bézier via les polynôme de Berstein


On admet la relation suivante entre les polynômes de la base de Bernstein $B_i ^n$, $i=0,\dots,n$ et la base canonique de $\mathbb{R}_n[x]$, $1,x,x^2,\dots,x^n$.
<table style="background: white;">
    <tr style="background: white;">
        <td> <img src='bernstein.svg'  width="500px">       
    </tr>
</table>

Etant donné un polygone de contrôle ${\bf P}=(P_0,\dots,P_n)$ et un temps $t\in [0,1]$, écrire deux fonctions qui expriment les polynômes $x(t)$ et $y(t)$ dans la base canonique (votre fonction pourra produire un vecteur pour les coefficients de $x$ et un vecteur pour les coefficients de $y$).

Utiliser cette construction pour tracer la courbe paramétrée $\gamma(t)=(x(t),y(t)$ en utilisant l'algorithme de Horner.

Comparer les temps d'exécution du tracé avec la méthode de de Casteljau.

## Fonction B-splines

Ecrire une fonction qui étant donnée un vecteur noeud $\tau=(t_0,\dots, t_m)$ et le choix d'un degré $k$ calcule les fonction B-splines $B_{i,k}$ pour $k=0,\dots,m-k-1$.

Donner des représentations graphiques des fonctions B-splines associées à un noeuds donné